In [1]:
import time
import numpy as np
import types
import os
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy.special import zeta
import PRyM.PRyM_init as PRyMini
import PRyM.PRyM_thermo as PRyMthermo
import PRyM.test_main3 as test_main
#import PRyM.PRyM_jl_sys as PRyMjl
from numdifftools import Derivative
import PRyM.PRyM_eval_nTOp as PRyMevalnTOp
import PRyM.PRyM_nTOp as PRyMnTOp
import matplotlib.pyplot as plt
import pandas as pd
import random

In [78]:
#test
'''params = [7.58363205, 2.91918682, 1.12641116, 0.03422378, 0.77294929, 4.11649565, 4.        ]
params = [ 6.01481611,  1.37305974  2.51980166  0.79767413  0.66060306  6.15616
 12.        ]
PRyMini.compute_bckg_flag = True
PRyMini.save_bckg_flag = False
PRyMini.stasis_flag = True
PRyMini.verbose_flag = False
PRyMini.stasis_params['gamma'] = params[0]
PRyMini.stasis_params['delta'] = params[1]
#PRyMini.stasis_params['alpha'] = params[2]
PRyMini.stasis_params['alpha'] = (2/7)*PRyMini.stasis_params['gamma'] - (1/PRyMini.stasis_params['delta'])
PRyMini.stasis_params['Delta_m'] = params[2]
PRyMini.stasis_params['m0'] = params[2]
PRyMini.stasis_params['plasma_inject_frac'] = params[3]
PRyMini.stasis_params['tiny_radiation'] = params[4]
PRyMini.stasis_params['stasis_end_mev'] = params[5]
PRyMini.stasis_params['N_SPECIES'] = int(params[6])
#print(PRyMini.stasis_params)
#predictions = model(params)
#residuals = (predictions - X_obs) / errors
#sum(((X_th-X_obs)/sigma_X)**2)
#calculate theoretical values
results = test_main.PRyMclass().PRyMresults()
Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10 = results'''

"params = [7.58363205, 2.91918682, 1.12641116, 0.03422378, 0.77294929, 4.11649565, 4.        ]\nparams = [ 6.01481611,  1.37305974  2.51980166  0.79767413  0.66060306  6.15616\n 12.        ]\nPRyMini.compute_bckg_flag = True\nPRyMini.save_bckg_flag = False\nPRyMini.stasis_flag = True\nPRyMini.verbose_flag = False\nPRyMini.stasis_params['gamma'] = params[0]\nPRyMini.stasis_params['delta'] = params[1]\n#PRyMini.stasis_params['alpha'] = params[2]\nPRyMini.stasis_params['alpha'] = (2/7)*PRyMini.stasis_params['gamma'] - (1/PRyMini.stasis_params['delta'])\nPRyMini.stasis_params['Delta_m'] = params[2]\nPRyMini.stasis_params['m0'] = params[2]\nPRyMini.stasis_params['plasma_inject_frac'] = params[3]\nPRyMini.stasis_params['tiny_radiation'] = params[4]\nPRyMini.stasis_params['stasis_end_mev'] = params[5]\nPRyMini.stasis_params['N_SPECIES'] = int(params[6])\n#print(PRyMini.stasis_params)\n#predictions = model(params)\n#residuals = (predictions - X_obs) / errors\n#sum(((X_th-X_obs)/sigma_X)**2)\n#

In [4]:
#observed values, all wrong currently, for testing purposes
#Neff_obs, Omeganurel_obs, OneOverOmeganunr_obs, YPCMB_obs, YPBBN_obs, DoHx1e5_obs, He3oHx1e5_obs, Li7oHx1e10_obs = np.array([3.04439907, 5.69942692, 93.03336616, 0.24552507, 0.24685084, 2.5276775, 1.04572838, 5.27256451])
#Yp, Omega_B h^2, Neff
#YPCMB_obs, OmegaB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs
YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs = np.array([0.245,3.85,2.547,1.63])
X_obs = np.array([YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs])
#X_obs = np.array([Neff_obs, Omeganurel_obs, OneOverOmeganunr_obs, YPCMB_obs, YPBBN_obs, DoHx1e5_obs, He3oHx1e5_obs, Li7oHx1e10_obs])
#Neff_sig, Omeganurel_sig, OneOverOmeganunr_sig, YPCMB, YPBBN_sig, DoHx1e5_sig, He3oHx1e5_sig, Li7oHx1e10_sig = np.array([0.5,0.5,3,.1,.1,.3,.2,1])
YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig = np.array([0.003,0.62,0.0029,0.3])
#sigma_X = np.array([Neff_sig, Omeganurel_sig, OneOverOmeganunr_sig, YPCMB, YPBBN_sig, DoHx1e5_sig, He3oHx1e5_sig, Li7oHx1e10_sig])
sigma_X = np.array([YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig])

In [3]:
'''filename = "output.txt"

# Create the file (or clear it if it already exists)
with open(filename, 'w') as f:
    f.write("Starting output:\n")'''

'filename = "output.txt"\n\n# Create the file (or clear it if it already exists)\nwith open(filename, \'w\') as f:\n    f.write("Starting output:\n")'

In [ ]:
# Chi^2 function
#params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]
#new params = [0    ,   1  ,  2  ,   3  , 4,  5  ,   6     ]
#new params = [gamma, delta, delm, fplas, f, Tend, Nspecies] (only independent parameters)
def chi2_fn(params):
    #initialize stasis parameters
    PRyMini.compute_bckg_flag = True
    PRyMini.save_bckg_flag = False
    PRyMini.stasis_flag = True
    PRyMini.verbose_flag = False
    PRyMini.stasis_params['gamma'] = params[0]
    PRyMini.stasis_params['delta'] = params[1]
    PRyMini.stasis_params['delta'] = 1 #test
    #PRyMini.stasis_params['alpha'] = params[2]
    PRyMini.stasis_params['alpha'] = (2/7)*PRyMini.stasis_params['gamma'] - (1/PRyMini.stasis_params['delta'])
    PRyMini.stasis_params['Delta_m'] = params[2]
    PRyMini.stasis_params['m0'] = params[2]
    PRyMini.stasis_params['Delta_m'] = .5    #test
    PRyMini.stasis_params['m0'] = .5 #test
    PRyMini.stasis_params['plasma_inject_frac'] = params[3]
    PRyMini.stasis_params['tiny_radiation'] = params[4]
    PRyMini.stasis_params['stasis_end_mev'] = params[5]
    PRyMini.stasis_params['N_SPECIES'] = int(params[6])
    #print(PRyMini.stasis_params)
    #predictions = model(params)
    #residuals = (predictions - X_obs) / errors
    #sum(((X_th-X_obs)/sigma_X)**2)
    #calculate theoretical values
    results = test_main.PRyMclass().PRyMresults()
    Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10 = results
    #X_th = np.array([Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10])
    X_th = np.array([YPCMB, Neff, DoHx1e5, Li7oHx1e10])
    return sum(((X_th-X_obs)/sigma_X)**2)

# Gradient of chi^2 (numerical)
def grad_chi2(chi2_fn,params, eps=1e-3):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        dparams = np.zeros_like(params)
        dparams[i] = eps
        grad[i] = (chi2_fn(params + dparams) - chi2_fn(params - dparams)) / (2 * eps)
        if i==2:
            print('mass before: ',params[i],' mass after: ',params[i]+dparams[i],'. These should be different. gradient: ',grad[i])
            print('chisq(p+dp) = ',chi2_fn(params + dparams),'chisq(p-dp) = ',chi2_fn(params - dparams),' eps = ',eps)
    return grad


In [6]:
#adaptive gradient method
def adagrad_minimize(chi2_fn, params_init, lr=0.0001, epsilon=1e-8, max_iters=1000, tol=1e-6, verbose=True):
    params = np.array(params_init, dtype=np.float64)
    G = np.zeros_like(params)
    
    for t in range(1, max_iters + 1):
        grad = grad_chi2(chi2_fn, params)
        G += grad ** 2
        adjusted_grad = grad / (np.sqrt(G) + epsilon)
        update = lr * adjusted_grad
        params -= update
        params[-1] = int(np.floor(params[-1]))#N_species needs to be an integer
        learning_rate = lr / (np.sqrt(G) + epsilon)
        if verbose:
            print(f"Iter {t}: params = {params}, chisq = {chi2_fn(params):.6f}, grad = {grad}, |grad| = {np.linalg.norm(grad):.3e}, learning rate*grad = {update}")
        
        if np.linalg.norm(update) < tol:
            break
            
    return params

In [6]:
PRyMini.stasis_params

{'gamma': 7,
 'delta': 1,
 'm0': 1.0,
 'Delta_m': 1.0,
 'N_SPECIES': 10,
 'GAMMA_OVER_H': 100000.0,
 'stasis_end_mev': 50,
 'tiny_radiation': 0.01,
 'plasma_inject_frac': 0.1,
 'alpha': 1.0}

In [ ]:
#run stasis gradient descent and save results ot a file
# Optional: write header only once
file_name = "results_grad_descent.txt"
with open(file_name, "w") as ff:
    ff.write("gamma delta alpha delm m0 fplas f Tend Nspecies chi2\n")
#initialize random stasis parameters in the desired intervals
gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
#print(Nspecies)
#initialize params
params = [gamma,delta,delm,fplas,f,Tend,Nspecies]
#params = [ 6.01481611,  1.37305974,  2.51980166,  0.79767413,  0.66060306,  6.15616, 12.        ]
#params = [7.926336094913086, 2.7753513891062145, 1.1264111582850547, 0.3486721409578998, 0.5688199257967876, 4.064037499639412, 4]
params = np.array(params)
print(params)
'''chi2 = chi2_fn(params)
with open(file_name, "a") as ff:
    ff.write(f"{gamma} {delta} {alpha} {delm} {m0} {fplas} {f} {Tend} {Nspecies} {chi2}\n")
#apply gradient descent, extremely slow right now
for n in range(1000):
    params = gradient_descent(params, lr=0.001, tol=7, max_iter=100)
    #chi2 = chi2_fn(params)
    with open(file_name, "a") as ff:
        ff.write(f"{params[0]} {params[1]} {params[2]} {params[3]} {params[4]} {params[5]} {params[6]} {params[7]} {params[8]} {chi2}\n")
        ff.flush()'''

adagrad_minimize(chi2_fn, params, lr=0.1, epsilon=1e-4, max_iters=1000, tol=1e-6, verbose=True)

[5.88336444394238, 1.4768581347450591, 1.646106938054658, 0.24917206959458524, 0.2203515833636362, 5.690066913087591, 10]
Iter 1: params = [5.78336444 1.57685813 1.74610694 0.14917207 0.12035158 5.59006693
 9.        ], chisq = 173.803866, grad = [  78381.39219762  -80260.64844858 -145853.79757972   46644.42020133
   38214.28316519     710.87237173  132984.91519115], |grad| = 2.349e+05, learning rate*grad = [ 0.1        -0.1        -0.1         0.1         0.1         0.09999999
  0.1       ]
Iter 2: params = [5.88310348 1.47790939 1.65045284 0.24855555 0.02068072 5.49006699
 9.        ], chisq = 206.507118, grad = [-1082816.10036051   549146.09996453   478450.07355795  -418113.49505079
   469835.64239128   644172.54096572 -2291599.22380807], |grad| = 2.787e+06, learning rate*grad = [-0.09973903  0.09894874  0.09565409 -0.09938348  0.09967086  0.09999994
 -0.09983204]
Iter 3: params = [5.88319103 1.53522245 1.71192014 0.18228305 0.0693478  5.48125645
 8.        ], chisq = 220.100374, g

/dfs6/pub/ahewitt1/Stasis/Stasis_prym/PRyM/test_main3.py:830: RuntimeWarning: invalid value encountered in sqrt
  return PRyMini.MeV_to_secm1 * np.sqrt(8*np.pi*PRyMini.GN/3 * rho_total)
/dfs6/pub/ahewitt1/Stasis/Stasis_prym/PRyM/test_main3.py:782: RuntimeWarning: invalid value encountered in log
  "chi_ell0":   np.log(rho_ell0),


ValueError: All components of the initial state `y0` must be finite.

In [ ]:
'''gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
#print(Nspecies)
#initialize params
params = [gamma,delta,delm,fplas,f,Tend,Nspecies]
chi2_fn(params)'''

KeyboardInterrupt: 

In [ ]:
#minimize -2 ln L with a package
from scipy.optimize import minimize
import numpy as np

'''#initialize random stasis parameters in the desired intervals
gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
print(Nspecies)
#initialize params
params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]

# Initial guess
#x0 = np.array([0.0, 0.0])

# Minimize
res = minimize(chi2_fn, params, method='BFGS')'''

19
{'gamma': np.float64(4.733232563550961), 'delta': np.float64(2.413703625087335), 'm0': np.float64(8.72149004071636), 'Delta_m': np.float64(8.72149004071636), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(1.3619708154105123), 'tiny_radiation': np.float64(0.9053552727974801), 'plasma_inject_frac': np.float64(0.22748234321992722), 'alpha': np.float64(0.9380510887510518)}
{'gamma': np.float64(4.733232578452122), 'delta': np.float64(2.413703625087335), 'm0': np.float64(8.72149004071636), 'Delta_m': np.float64(8.72149004071636), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(1.3619708154105123), 'tiny_radiation': np.float64(0.9053552727974801), 'plasma_inject_frac': np.float64(0.22748234321992722), 'alpha': np.float64(0.9380510887510518)}
{'gamma': np.float64(4.733232563550961), 'delta': np.float64(2.4137036399884964), 'm0': np.float64(8.72149004071636), 'Delta_m': np.float64(8.72149004071636), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, '

KeyboardInterrupt: 